In [ ]:
# Group members: Lilian Yu (jiajieyu), Ziyi Shen (ziyis), Tristan Zhang (jinyang3)

In [ ]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
# import project_helper

from tqdm import tqdm

In [ ]:
# project_helper

import matplotlib.pyplot as plt
import requests

from ratelimit import limits, sleep_and_retry


class SecAPI(object):
    SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}

    @staticmethod
    @sleep_and_retry
    # Dividing the call limit by half to avoid coming close to the limit
    @limits(calls=SEC_CALL_LIMIT['calls'] / 2, period=SEC_CALL_LIMIT['seconds'])
    def _call_sec(url):
        return requests.get(url)

    def get(self, url):
        return self._call_sec(url).text


def print_ten_k_data(ten_k_data, fields, field_length_limit=50):
    indentation = '  '

    print('[')
    for ten_k in ten_k_data:
        print_statement = '{}{{'.format(indentation)
        for field in fields:
            value = str(ten_k[field])

            # Show return lines in output
            if isinstance(value, str):
                value_str = '\'{}\''.format(value.replace('\n', '\\n'))
            else:
                value_str = str(value)

            # Cut off the string if it gets too long
            if len(value_str) > field_length_limit:
                value_str = value_str[:field_length_limit] + '...'

            print_statement += '\n{}{}: {}'.format(indentation * 2, field, value_str)

        print_statement += '},'
        print(print_statement)
    print(']')


def plot_similarities(similarities_list, dates, title, labels):
    assert len(similarities_list) == len(labels)

    plt.figure(1, figsize=(10, 7))
    for similarities, label in zip(similarities_list, labels):
        plt.title(title)
        plt.plot(dates, similarities, label=label)
        plt.legend()
        plt.xticks(rotation=90)

    plt.show()

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
#cik_lookup = {
  #  'AMZN': '0001018724',
 #   'BMY': '0000014272',   
  #  'CNP': '0001130310',
  #  'CVX': '0000093410',
 #   'FL': '0000850209',
 #   'FRT': '0000034903',
 #   'HON': '0000773840'}
#cik_lookup = {
  # 'AMZN': '0001018724',
 #   'BMY': '0000014272'}

In [ ]:
#import tick and cik
#from google.colab import files
#uploaded = files.upload()

#import io
df2 = pd.read_csv('sp500_w_addl_id_with_cik.csv')
# Dataset is now stored in a Pandas Dataframe

In [ ]:
df = df2[['date','ticker', 'cik']]
df=df.loc[df['date']>='2017-01-01']
df

In [ ]:
tickers = df.groupby('date')['ticker'].apply(list)
ciks = df.groupby('date')['cik'].apply(list)
tickers,ciks
#sp500=df.groupby('ticker')['date'].apply(list)
#sp500

In [ ]:
#tickers.keys()
#
#df.loc[df['cik'].isna()] AA and DPS have no cik listed
df = df[df.cik.notnull()]
#df['cik'].astype(int)
#cik_lookup=dict(zip(df.ticker[:5], df['cik'][:5].astype(int)))
#cik_lookup                                                          ##pick first 5

In [ ]:
look= df.groupby('ticker')['cik'].apply(list)
look=look.to_dict()
#lookup=dict(zip(look.ticker, look['ticker'][0].astype(int)))
#lookup
cik_lookupss={}
for ticker, ciks in look.items():
    cik_lookupss[ticker]=int(ciks[0])
##cik_lookupss.keys()     ##656 companies

In [ ]:
dates = df2['date'].unique()

In [ ]:
ALL_ticker=list(cik_lookupss.keys())

In [ ]:
########
num = 450
cik_lookup= { key: cik_lookupss[key] for key in ALL_ticker[num:(num+50)] }   #first 50 comps
#cik_lookup

In [ ]:
######10K

sec_api = SecAPI()

from bs4 import BeautifulSoup

HEADER = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }
def get_sec_data(cik, doc_type, start=0, count=60):
  rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
      '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
      .format(cik, doc_type, start, count)
  sec_data = requests.get(rss_url,headers=HEADER)
  feed = BeautifulSoup(sec_data.content, 'html')
  entries = [
      (
          entry.content.find('filing-href').getText(),
          entry.content.find('filing-type').getText(),
          entry.content.find('filing-date').getText())
      for entry in feed.find_all('entry')]
  return entries

example_ticker = ALL_ticker[num]
sec_data = {}   #dict 10K
for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik, '10-K')[0:5] ##
pprint.pprint(sec_data[example_ticker])

In [ ]:
#####10Q

sec_api = SecAPI()

from bs4 import BeautifulSoup

HEADER = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }
def get_sec_data(cik, doc_type, start=0, count=60):
  rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
      '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
      .format(cik, doc_type, start, count)
  sec_data = requests.get(rss_url,headers=HEADER)
  feed = BeautifulSoup(sec_data.content, 'html')
  entries = [
      (
          entry.content.find('filing-href').getText(),
          entry.content.find('filing-type').getText(),
          entry.content.find('filing-date').getText())
      for entry in feed.find_all('entry')]
  return entries

example_ticker = ALL_ticker[num]
sec_data_10Q = {}     #dict 10Q
for ticker, cik in cik_lookup.items():
    sec_data_10Q[ticker] = get_sec_data(cik, '10-Q')[2:18]
pprint.pprint(sec_data_10Q[example_ticker])

In [ ]:
#periods=pd.DataFrame()
#periods['End']=tickers.keys()
#periods['Start']=periods.End.shift(1)
#periods['Start'][0]='2017-01-01'
#periods.head()

In [ ]:
#for ticker in cik_lookup.keys():
#  print(ticker)

In [ ]:
for ticker in cik_lookup.keys():     ##10K  keep all in the time horizon 5 years
  for item in sec_data[ticker]:
    if (item[-1]<'2017-01-03') or (item[-1]>'2021-12-31'):
      sec_data[ticker].remove(item)

In [ ]:
for ticker in cik_lookup.keys():   
  for item in sec_data_10Q[ticker]:     ##10Q  keep all in the time horizon
    if (item[-1]<'2017-01-03') or (item[-1]>'2021-12-31'):
      sec_data_10Q[ticker].remove(item)

In [ ]:
#sec_data_10Q[ticker]  Now all 10Q files are in date range

In [ ]:
####Take long time, could play around for a while


raw_fillings_by_ticker = {}     
for ticker, data in sec_data.items():   ###get 10K from dict
    try:
        raw_fillings_by_ticker[ticker] = {}   ###create empty 10K filing
        for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
            if (file_type == '10-K'):
                file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
                raw_fillings_by_ticker[ticker][file_date] = requests.get(file_url,headers=HEADER).text
    except:
        print(ticker)
print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

In [ ]:
import re
def get_documents(text):
    extracted_docs = []
    
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')   
    
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(text)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
    
    for doc_start_i, doc_end_i in zip(doc_start_is, doc_end_is):
            extracted_docs.append(text[doc_start_i:doc_end_i])
    
    return extracted_docs

filling_documents_by_ticker = {}
for ticker, raw_fillings in raw_fillings_by_ticker.items():
    try:
        filling_documents_by_ticker[ticker] = {}
        for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
            filling_documents_by_ticker[ticker][file_date] = get_documents(filling)
    except:
        print(ticker)
print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

In [ ]:
def get_document_type(doc):
    
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    
    doc_type = type_pattern.findall(doc)[0][len('<TYPE>'):] 
    
    return doc_type.lower()

In [ ]:
ten_ks_by_ticker = {}
for ticker, filling_documents in filling_documents_by_ticker.items():
    try:
        ten_ks_by_ticker[ticker] = []
        for file_date, documents in filling_documents.items():
            for document in documents:
                if get_document_type(document) == '10-k':
                    ten_ks_by_ticker[ticker].append({
                        'cik': cik_lookup[ticker],
                        'file': document,
                        'file_date': file_date})
    except:
        print(ticker)
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])

In [ ]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text
def clean_text(text):
    text = text.lower().strip()
    text = remove_html_tags(text)
    #text = re.sub("[^a-zA-Z]"," ",text).split(' ')  #this will take off the release date, do it later before 
    
    return text



In [ ]:
####Take long time, could play around for a while  ~10min for 50 comps 10K

for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_clean'] = clean_text(ten_k['file'])
    except:
        print(ticker)
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean'])

In [ ]:
#print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_date'])
print_ten_k_data(ten_ks_by_ticker[example_ticker][0:1], ['file_clean'])

In [ ]:
ten_ks_by_ticker[example_ticker][0]['file_date']  ##the 0th company's file date

lemmatize all the data

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def lemmatize_words(words):

    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]
    
    return lemmatized_words
word_pattern = re.compile('\w+')
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Lemmatize {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_lemma'] = lemmatize_words(word_pattern.findall(ten_k['file_clean']))
    except:
        print(ticker)
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_lemma'])

Remove the stopwords.

In [ ]:
from nltk.corpus import stopwords
lemma_english_stopwords = lemmatize_words(stopwords.words('english'))
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Remove Stop Words for {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_lemma'] = [word for word in ten_k['file_lemma'] if word not in lemma_english_stopwords]
    except:
        print(ticker)
print('Stop Words Removed')

In [ ]:
ten_k['file_lemma']

In [ ]:
###take out strings containing numbers

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def clean_numb(text):
    
    text = [item for subitem in text for item in subitem.split() if item.isdigit()==False]  
    text = [item for subitem in text for item in subitem.split() if has_numbers(item)==False]
    return text


for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
            ten_k['file_clean_numb'] = clean_numb(ten_k['file_lemma'])
    except:
        print
print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean_numb'])

In [ ]:
ten_ks_by_ticker[example_ticker][0]['file_clean_numb']

Deal with 10Q the same way

In [ ]:
#Take long time  11:58-12:13  ~15min for 50comps
raw_fillings_by_ticker_10Q = {}     
for ticker, data in sec_data_10Q.items():   ###get 10Q from dict
    try:
        raw_fillings_by_ticker_10Q[ticker] = {}   ###create empty 10Q filing
        for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
            if (file_type == '10-Q'):
                file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
                raw_fillings_by_ticker_10Q[ticker][file_date] = requests.get(file_url,headers=HEADER).text
    except:
        print(ticker)
print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker_10Q[example_ticker].values()))[:1000]))

In [ ]:
filling_documents_by_ticker_10Q = {}
for ticker, raw_fillings in raw_fillings_by_ticker_10Q.items():
    try:
        filling_documents_by_ticker_10Q[ticker] = {}
        for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
            filling_documents_by_ticker_10Q[ticker][file_date] = get_documents(filling)
    except:
        print(ticker)
print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker_10Q[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

In [ ]:
#1 min

ten_Qs_by_ticker = {}
for ticker, filling_documents in filling_documents_by_ticker_10Q.items():
    ten_Qs_by_ticker[ticker] = []
    for file_date, documents in filling_documents.items():
        for document in documents:
            if get_document_type(document) == '10-q':
                ten_Qs_by_ticker[ticker].append({
                    'cik': cik_lookup[ticker],
                    'file': document,
                    'file_date': file_date})
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])    ###use print_ten_k_data to print 10Q

In [37]:

#Take long time   1:35pm-1:58  20min or 50comps

##slice_tenQ=list(ten_Qs_by_ticker.keys())
##ten_Qs_by_ticker_sliced= {key: ten_Qs_by_ticker[key] for key in slice_tenQ[:9] }    #ADBE is indexed-9

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Cleaning {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_clean'] = clean_text(ten_Q['file'])
    except:
        print(ticker)
        #del ten_Qs_by_ticker[ticker]
        
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['file_clean'])

#enountered error at ADBE, so skipped ADBE's 10Q 
#next time encounter error just skip and print tickers

Cleaning SBUX 10-Qs:  53%|█████▎    | 8/15 [00:14<00:13,  1.96s/10-Q]

In [ ]:
ten_Qs_by_ticker.keys()

In [ ]:
word_pattern = re.compile('\w+')
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Lemmatize {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_lemma'] = lemmatize_words(word_pattern.findall(ten_Q['file_clean']))
    except:
        print(ticker)
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['file_lemma'])

In [ ]:
from nltk.corpus import stopwords
lemma_english_stopwords = lemmatize_words(stopwords.words('english'))
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Remove Stop Words for {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_lemma'] = [word for word in ten_Q['file_lemma'] if word not in lemma_english_stopwords]
    except:
        print(ticker)
print('Stop Words Removed')

In [ ]:
def clean_numbQ(text):
    
    text = [item for subitem in text for item in subitem.split() if item.isdigit()==False]  
    text = [item for subitem in text for item in subitem.split() if has_numbers(item)==False]
    return text

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    try:
        for ten_Q in tqdm(ten_Qs, desc='Cleaning {} 10-Qs'.format(ticker), unit='10-Q'):
            ten_Q['file_clean_numb'] = clean_numbQ(ten_Q['file_lemma'])
    except:
        print(ticker)
print_ten_k_data(ten_Qs_by_ticker[example_ticker][:5], ['file_clean_numb'])

In [ ]:
sentiments = ['negative']

Fin_Neg=pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
#Fin_Neg=Fin_Neg.loc[Fin_Neg['Negative']>=2009]


sentiment_df = Fin_Neg
sentiment_df.columns = [column.lower() for column in sentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
sentiment_df = sentiment_df[sentiments + ['word']]
sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')


sentiment_df.head()

In [ ]:
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
#import project_helper
def get_bag_of_words(sentiment_words, docs):

    vec = CountVectorizer(vocabulary=sentiment_words)
    vectors = vec.fit_transform(docs)
    words_list = vec.get_feature_names()
    bag_of_words = np.zeros([len(docs), len(words_list)])
    
    for i in range(len(docs)):
        bag_of_words[i] = vectors[i].toarray()[0]
    return bag_of_words.astype(int)

In [ ]:
sentiment_bow_ten_ks = {}   #####setup ONCE?

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_clean_numb']) for ten_k in ten_ks]

    sentiment_bow_ten_ks[ticker] = {sentiment: get_bag_of_words(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}


In [ ]:
def get_negative_word_countK(ticker):
    return np.cumsum(sentiment_bow_ten_ks[ticker]['negative'], axis=1)[:,-1]


def get_total_word_countK(ticker):
    total=[0]*len(ten_ks_by_ticker[ticker])
    for i in range(len(ten_ks_by_ticker[ticker])):
        total[i]=len(ten_ks_by_ticker[ticker][i]['file_clean_numb'])
    return total   

In [ ]:
FNproporK={}
for ticker in ten_ks_by_ticker.keys():
    FNproporK[ticker]=get_negative_word_countK(ticker)/get_total_word_countK(ticker)

In [ ]:
def get_file_dateK(ticker):
    fdate=[0]*len(ten_ks_by_ticker[ticker])
    for i in range(len(ten_ks_by_ticker[ticker])):
        fdate[i]=ten_ks_by_ticker[ticker][i]['file_date']
    return fdate

fdatek={}
for ticker in ten_ks_by_ticker.keys():
    fdatek[ticker]=get_file_dateK(ticker)

In [ ]:
def get_FNproporK_date(fdatek,proporK):
    FNproporKDATE={}
    for ticker in fdatek.keys():
        FNproporK_date= {fdatek[ticker][i]: proporK[ticker][i] for i in range(len(proporK[ticker]))}
        FNproporKDATE.update(FNproporK_date)
    return FNproporKDATE

In [ ]:
FN_10K_portion=get_FNproporK_date(fdatek,FNproporK)    ###date: proportion of 10K negaword FinNeg
#AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAaaaaaaaaaa

In [ ]:
colname=['Word'] 
H4N=pd.read_csv('Harvard IV_Negative Word List_Inf.txt',names=colname,header=None)

H4N['negative']=True
#H4N

In [ ]:
sentiments = ['negative']

#Fin_Neg=pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
#Fin_Neg=Fin_Neg.loc[Fin_Neg['Negative']>=2009]


Hsentiment_df = H4N
Hsentiment_df.columns = [column.lower() for column in Hsentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
Hsentiment_df = Hsentiment_df[sentiments + ['word']]
Hsentiment_df[sentiments] = Hsentiment_df[sentiments].astype(bool)
Hsentiment_df = Hsentiment_df[(Hsentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
Hsentiment_df['word'] = lemmatize_words(Hsentiment_df['word'].str.lower())
Hsentiment_df = Hsentiment_df.drop_duplicates('word')


Hsentiment_df.head()

In [ ]:
Hsentiment_bow_ten_ks = {}   #####setup ONCE?

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_clean_numb']) for ten_k in ten_ks]

    Hsentiment_bow_ten_ks[ticker] = {sentiment: get_bag_of_words(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}

In [ ]:
def Hget_negative_word_countK(ticker):
    return np.cumsum(Hsentiment_bow_ten_ks[ticker]['negative'], axis=1)[:,-1]


HproporK={}
for ticker in ten_ks_by_ticker.keys():
    HproporK[ticker]=Hget_negative_word_countK(ticker)/get_total_word_countK(ticker)

In [ ]:
def get_HproporK_date(fdatek,HproporK):
    HproporKDATE={}
    for ticker in fdatek.keys():
        HproporK_date= {fdatek[ticker][i]: HproporK[ticker][i] for i in range(len(HproporK[ticker]))}
        HproporKDATE.update(HproporK_date)
    return HproporKDATE

In [ ]:
H_10K_portion=get_HproporK_date(fdatek,HproporK) 

In [ ]:
def get_file_dateQ(ticker):
    fdate=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        fdate[i]=ten_Qs_by_ticker[ticker][i]['file_date']
    return fdate

fdateQ={}
for ticker in ten_Qs_by_ticker.keys():
    fdateQ[ticker]=get_file_dateQ(ticker)

In [ ]:
sentiment_bow_ten_Qs = {}   #####setup ONCE?

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_clean_numb']) for ten_Q in ten_Qs]

    sentiment_bow_ten_Qs[ticker] = {sentiment: get_bag_of_words(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}

print_ten_k_data([sentiment_bow_ten_Qs[example_ticker]], sentiments)

In [ ]:
def get_negative_word_countQ(ticker):
    return np.cumsum(sentiment_bow_ten_Qs[ticker]['negative'], axis=1)[:,-1]

def get_total_word_countQ(ticker):
    total=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        total[i]=len(ten_Qs_by_ticker[ticker][i]['file_clean_numb'])
    return total   

In [ ]:
FNproporQ={}
for ticker in ten_Qs_by_ticker.keys():
    FNproporQ[ticker]=get_negative_word_countQ(ticker)/get_total_word_countQ(ticker)
#get_total_word_count('A')
len(FNproporQ)

In [ ]:
def get_file_dateQ(ticker):
    fdate=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        fdate[i]=ten_Qs_by_ticker[ticker][i]['file_date']
    return fdate

fdateQ={}
for ticker in ten_Qs_by_ticker.keys():
    fdateQ[ticker]=get_file_dateQ(ticker)

In [ ]:
def get_FNproporQ_date(fdateQ,proporQ):
    FNproporQDATE={}
    for ticker in fdateQ.keys():
        FNproporQ_date= {fdateQ[ticker][i]: proporQ[ticker][i] for i in range(len(proporQ[ticker]))}
        FNproporQDATE.update(FNproporQ_date)
    return FNproporQDATE

In [ ]:
FN_10Q_portion=get_FNproporQ_date(fdateQ,FNproporQ)  

In [ ]:
Hsentiment_bow_ten_Qs = {}   #####setup ONCE?

for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_clean_numb']) for ten_Q in ten_Qs]

    Hsentiment_bow_ten_Qs[ticker] = {sentiment: get_bag_of_words(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
       for sentiment in sentiments}

print_ten_k_data([Hsentiment_bow_ten_Qs[example_ticker]], sentiments)

In [ ]:
def Hget_negative_word_countQ(ticker):
    return np.cumsum(Hsentiment_bow_ten_Qs[ticker]['negative'], axis=1)[:,-1]

HproporQ={}
for ticker in ten_Qs_by_ticker.keys():
    HproporQ[ticker]=Hget_negative_word_countQ(ticker)/get_total_word_countQ(ticker)
    
def get_HproporQ_date(fdateQ,HproporQ):
    HproporQDATE={}
    for ticker in fdateQ.keys():
        HproporQ_date= {fdateQ[ticker][i]: HproporQ[ticker][i] for i in range(len(HproporQ[ticker]))}
        HproporQDATE.update(HproporQ_date)
    return HproporQDATE

In [ ]:
H_10Q_portion=get_HproporQ_date(fdateQ,HproporQ)
H_10Q_portion

In [ ]:
###TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
def get_tfidf(sentiment_words, docs):
    
    vec = TfidfVectorizer(vocabulary=sentiment_words)
    tfidf = vec.fit_transform(docs)
    
    return tfidf.toarray()

In [ ]:
##10K tfidf   FN

FNsentiment_tfidf_ten_ks = {}
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    
        FNsentiment_tfidf_ten_ks[ticker] = {
            sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
            for sentiment in sentiments}
        
    except:
        print(ticker)
print_ten_k_data([FNsentiment_tfidf_ten_ks[example_ticker]], sentiments)

In [ ]:
FNsentiment_tfidf_ten_ks['OGN']={'negative':np.empty([5,1522])}


In [ ]:
FN_tfidf_K={}
for ticker in ten_ks_by_ticker.keys():
    array=np.cumsum(FNsentiment_tfidf_ten_ks[ticker]['negative'], axis=1)[:,-1]
    FN_tfidf_K[ticker]=array
#FN_tfidf_K                                       ####### FN tfidf for 10K 

In [ ]:
FN_tfidf_K_DATE={}
for ticker in fdatek.keys():
    try:
        FNproporK_date= {fdatek[ticker][i]: FN_tfidf_K[ticker][i] for i in range(len(FN_tfidf_K[ticker]))}
        FN_tfidf_K_DATE.update(FNproporK_date)
    except:
        # FNproporK_date=
        print(ticker)
FN_tfidf_K_DATE   

In [ ]:
##10K tfidf   H4N

Hsentiment_tfidf_ten_ks = {}
for ticker, ten_ks in ten_ks_by_ticker.items():
    try:
        lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    
        Hsentiment_tfidf_ten_ks[ticker] = {
            sentiment: get_tfidf(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
            for sentiment in sentiments}
    except:
        print(ticker)
#print_ten_k_data([FNsentiment_tfidf_ten_ks[example_ticker]], sentiments)


H_tfidf_K={}
for ticker in ten_ks_by_ticker.keys():
    try:
        array=np.cumsum(Hsentiment_tfidf_ten_ks[ticker]['negative'], axis=1)[:,-1]
        H_tfidf_K[ticker]=array
    except:
        print(ticker)
#H_tfidf_K         


H_tfidf_K_DATE={}
for ticker in fdatek.keys():
    try:
        HproporK_date= {fdatek[ticker][i]: H_tfidf_K[ticker][i] for i in range(len(H_tfidf_K[ticker]))}
        H_tfidf_K_DATE.update(HproporK_date)
    except:
        print(ticker)
H_tfidf_K_DATE 

In [ ]:
def get_file_dateQ(ticker):
    fdate=[0]*len(ten_Qs_by_ticker[ticker])
    for i in range(len(ten_Qs_by_ticker[ticker])):
        fdate[i]=ten_Qs_by_ticker[ticker][i]['file_date']
    return fdate

fdateQ={}
for ticker in ten_Qs_by_ticker.keys():
    fdateQ[ticker]=get_file_dateQ(ticker)

In [ ]:
##10Q tfidf   FN

FNsentiment_tfidf_ten_Qs = {}
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_lemma']) for ten_Q in ten_Qs]
    
    FNsentiment_tfidf_ten_Qs[ticker] = {
        sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}
#print_ten_k_data([FNsentiment_tfidf_ten_Qs[example_ticker]], sentiments)


FN_tfidf_Q={}
for ticker in ten_Qs_by_ticker.keys():
    array=np.cumsum(FNsentiment_tfidf_ten_Qs[ticker]['negative'], axis=1)[:,-1]
    FN_tfidf_Q[ticker]=array
#FN_tfidf_Q                                       ####### FN tfidf for 10Q 

FN_tfidf_Q_DATE={}
for ticker in fdateQ.keys():
    FNproporQ_date= {fdateQ[ticker][i]: FN_tfidf_Q[ticker][i] for i in range(len(FN_tfidf_Q[ticker]))}
    FN_tfidf_Q_DATE.update(FNproporQ_date)
FN_tfidf_Q_DATE                                                      ####### FN tfidf for 10Q 

In [ ]:
##10Q tfidf   H

Hsentiment_tfidf_ten_Qs = {}
for ticker, ten_Qs in ten_Qs_by_ticker.items():
    lemma_docs = [' '.join(ten_Q['file_lemma']) for ten_Q in ten_Qs]
    
    Hsentiment_tfidf_ten_Qs[ticker] = {
        sentiment: get_tfidf(Hsentiment_df[Hsentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}
#print_ten_k_data([Hsentiment_tfidf_ten_Qs[example_ticker]], sentiments)


H_tfidf_Q={}
for ticker in ten_Qs_by_ticker.keys():
    array=np.cumsum(Hsentiment_tfidf_ten_Qs[ticker]['negative'], axis=1)[:,-1]
    H_tfidf_Q[ticker]=array
#FN_tfidf_Q                                       ####### H tfidf for 10Q 

H_tfidf_Q_DATE={}
for ticker in fdateQ.keys():
    HproporQ_date= {fdateQ[ticker][i]: H_tfidf_Q[ticker][i] for i in range(len(H_tfidf_Q[ticker]))}
    H_tfidf_Q_DATE.update(HproporQ_date)
H_tfidf_Q_DATE                                                      ####### H tfidf for 10Q 

In [ ]:
c10k= [H_10K_portion, FN_10K_portion,H_tfidf_K_DATE,FN_tfidf_K_DATE]
K={}

for k in c10k[0].keys():
  K[k] = list(d[k] for d in c10k)

In [ ]:

c10Q= [H_10Q_portion,FN_10Q_portion,H_tfidf_Q_DATE,FN_tfidf_Q_DATE]
Q={}

for k in c10Q[0].keys():
  Q[k] = list(d[k] for d in c10Q)

In [ ]:
dfK = pd.DataFrame.from_dict(K).T
dfQ=pd.DataFrame.from_dict(Q).T


In [ ]:
frames = [dfK,dfQ]

DF0_50= pd.concat(frames)
DF0_50.columns = ['H4N_portion','FN_portion','H_tfidf','FN_tfidf']
DF0_50

In [ ]:
DF0_50.to_csv('DF_451_500.csv')

In [ ]:
from datetime import datetime

stock = pd.read_csv("stock master.csv",dtype = {'tic': str})
stock['datadate'] = pd.to_datetime(stock['datadate'], format='%Y%m%d')

spy_return = pd.read_csv("sp500 daily returns tristan.csv")
spy_return['caldt'] = pd.to_datetime(spy_return['caldt'], format='%Y%m%d')
vwr = spy_return['vwretd'] + 1
vwr1 = vwr.shift(1)
vwr2 = vwr.shift(2)

sst = -stock.groupby(['tic'])['prccd'].diff(-4)/stock['prccd']
datadic = {'date':stock['datadate'],'tic':stock['tic'], 'sst':sst}
stock_new = pd.DataFrame(datadic)

spy_return_ret = vwr*vwr1*vwr2-1
spy_return_ret = spy_return_ret.shift(-3)
datadic = {'date': spy_return['caldt'], 'idx': spy_return_ret}
spy_return_new = pd.DataFrame(datadic)

excess_return = stock_new.merge(spy_return_new, on = 'date')
excess_return = excess_return.dropna()
excess_return = excess_return.sort_values(by=['tic', 'date'])
excess_return['excess_return'] = excess_return['sst'] - excess_return['idx']

def get_return(ticker, date_str):
    year = date_str[:4] # '2021-12-17'
    month = date_str[5:7]
    day = date_str[8:10]
    # print(year, month, day)
    date = datetime(int(year), int(month), int(day))
    
    ret = excess_return[excess_return['tic']==ticker]
    ret = ret.drop_duplicates(subset=['date'])
    ret = ret.set_index('date')
    pos = ret.index.get_loc(date, method='bfill')
    ret = ret.iloc[pos]['excess_return']
    
    return ret


# print(excess_return['tic'])
#print(get_return('A', '2021-12-17'))

In [ ]:
excess_returnK={}
for ticker,dates in fdatek.items():
    for date in dates:
        try:
            excess_returnK[date]=get_return(ticker, str(date))
        except:
            excess_returnK[date]=0.0
#excess_returnK

In [ ]:
excess_returnQ={}
for ticker,dates in fdateQ.items():
    for date in dates:
        try:
            excess_returnQ[date]=get_return(ticker, str(date))
        except:
            excess_returnQ[date]=0.0
excess_returnQ

In [ ]:
df = pd.DataFrame.from_dict(excess_returnK,orient='index')
df2= pd.DataFrame.from_dict(excess_returnQ,orient='index')
df3 = df.append(df2)
DF0_50_c = DF0_50
DF0_50_c['return'] = df3
DF0_50_c.to_csv('DF0_' + str(num) + '.csv')